In [1]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Initialisation du driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()

# Identifiants de connexion
username = "nabiladouma23@gmail.com"
password = "uDB3Pc-q:T.rvNB"

# Accéder à la page de connexion
driver.get("https://app.chartmetric.com/login")
time.sleep(3)  # Attendre le chargement

# Connexion
try:
    driver.find_element(By.NAME, 'username').send_keys(username)
    driver.find_element(By.NAME, 'password').send_keys(password)
    driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]').click()
    print("Connexion réussie !")
except Exception as e:
    print(f"Erreur lors de la connexion : {e}")

time.sleep(10)  # Attendre après connexion

# URLs des plateformes à scraper
urls = {
    "applemusic": "https://app.chartmetric.com/fr/playlists/applemusic",
    "youtube": "https://app.chartmetric.com/fr/playlists/youtube",
    "deezer": "https://app.chartmetric.com/fr/playlists/deezer",
    "soundcloud": "https://app.chartmetric.com/fr/playlists/soundcloud",
    "amazon": "https://app.chartmetric.com/fr/playlists/amazon",
    "spotify": "https://app.chartmetric.com/fr/playlists/spotify"
}

# Liste pour stocker les résultats (éviter les doublons)
collected_playlists = set()
data = []

def scrape_playlists(platform, url):
    global collected_playlists, data
    print(f"Scraping {platform}...")

    driver.get(url)
    time.sleep(7)  # Attente du chargement initial

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        playlists = driver.find_elements(By.CSS_SELECTOR, 'span.Table_tableCell__yZJeq')

        for playlist in playlists:
            try:
                name = playlist.find_element(By.CSS_SELECTOR, 'a.EntityCover_name__6TCOr').text
                genres = [genre.text for genre in playlist.find_elements(By.CSS_SELECTOR, 'span.Cover_genreBadge__zm_87')]
                moods = [mood.text for mood in playlist.find_elements(By.CSS_SELECTOR, 'span.MoodTag_text__ivxkK')]

                # Ajouter uniquement les nouvelles playlists
                if name not in collected_playlists:
                    collected_playlists.add(name)
                    data.append([platform, name, ", ".join(genres), ", ".join(moods)])
            except Exception:
                continue

        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Vérifier si on est arrivé en bas de la page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Scraper chaque plateforme
for platform, url in urls.items():
    scrape_playlists(platform, url)

# Enregistrer les résultats dans un fichier CSV
filename = "chartmetric_playlists.csv"
with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['platform', 'name', 'genres', 'moods'])
    writer.writerows(data)

# Fermer le navigateur
driver.quit()

print(f"✅ Scraping terminé. {len(data)} playlists enregistrées dans '{filename}'")

Connexion réussie !
Scraping applemusic...
Scraping youtube...
Scraping deezer...
Scraping soundcloud...
Scraping amazon...
Scraping spotify...
✅ Scraping terminé. 218 playlists enregistrées dans 'chartmetric_playlists.csv'
